<a href="https://colab.research.google.com/github/alecseiterr/Tetrabot/blob/main/Andrey_Novikov/002_notebooks/transcribe_v2t_small_russian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Проверка распознавания голосовых команд записанных в файлы и произносимых с микрофона с помощью библиотеки vosk (компактная модель: vosk-model-small-ru-0.22 со страницы https://alphacephei.com/vosk/models

### Первая часть. Проверка распознавания голосовых команд из wav-файлов

In [1]:
# Установим библиотеки vosk и pydub
!pip install vosk pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.2 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22429 sha256=26170ca3cadd66a89e6c188ed909c2eb68776c33a24b39e9848b539e4815669c
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt


In [2]:
# Загрузим необходимые модули и библиотеки
import os
import sys
import json
from pydub import AudioSegment
from vosk import Model, KaldiRecognizer
from google.colab import drive
import shutil

In [3]:
# Примонтируем Google Drive
drive.mount('/content/drive')

Mounted at /content/drive



*   Скачаем малую модель vosk-model-small-ru-0.22 со страницы https://alphacephei.com/vosk/models
*   Загрузим малую модель в папку /content/drive/MyDrive/AI_data/tetrabot/models
*   Распакуем малую модель с помощью кода, приведенного ниже в папку /content/model

In [4]:
zip_file = '/content/drive/MyDrive/AI_data/tetrabot/models/vosk-model-small-ru-0.22.zip'
#zip_file = '/content/drive/MyDrive/AI_data/tetrabot/models/vosk-model-ru-0.42.zip'
extract_dir = '/content/model'
shutil.unpack_archive(zip_file, extract_dir)

In [5]:
# Укажем путь к модели
vosk_model_path = "/content/model/vosk-model-small-ru-0.22"
#vosk_model_path = '/content/model/vosk-model-ru-0.42'

# Загрузим модель
vosk_model = Model(vosk_model_path)

# Создадим экземпляр распознавателя голоса с помощью модели Vosk
recognizer = KaldiRecognizer(vosk_model, 16000)

In [68]:
# Функция распознавания голосовых команд из записанных файлов
def recognize_voice_command(audio_file_path):
    # Загрузка аудио файлов в виде AudioSegment
    audio = AudioSegment.from_wav(audio_file_path)

    # Преобразование записи в один канал с частотой дискретизации 16 kHz (требование модели Vosk)
    audio = audio.set_frame_rate(16000).set_channels(1)

    # Разпознавание голоса
    results = []
    chunk_size_ms = 4000  # Обработка потока чанками в 4 секунды
    for i in range(0, len(audio), chunk_size_ms):
        chunk = audio[i:i + chunk_size_ms]
        if len(chunk) > 0:
            raw_data = chunk.raw_data
            recognizer.AcceptWaveform(raw_data)
            result = json.loads(recognizer.Result())
            results.append(result)

    # Сборка распознанных фрагментов и вывод на экран
    recognized_text = " ".join(result['text'] for result in results)
    print("Распознанный текст: {:25}".format(recognized_text))


In [69]:
# Воспользуемся записями своего голоса (Вариант 1)
audio_folder_path = "/content/drive/MyDrive/AI_data/tetrabot/my_voice"

# Воспользуемся записями своего голосаn (Вариант 2)
#audio_folder_path = "/content/drive/MyDrive/AI_data/tetrabot/my_voice_Troy"

# Запись, предоставленная заказчиком
#audio_folder_path = "/content/drive/MyDrive/AI_data/tetrabot/tetrabot_new/golos_shagohod/golos_blizko"

In [71]:
# Запустим распознавание и транскрибирование всех файлов в заданной папке
for filename in os.listdir(audio_folder_path):
    if filename.endswith(".wav"):
        audio_file_path = os.path.join(audio_folder_path, filename)
        print("Процессинг файла  : {:25}".format(filename))
        recognize_voice_command(audio_file_path)

Процессинг файла  : направо.wav              
Распознанный текст: направо                  
Процессинг файла  : вправо.wav               
Распознанный текст: вправо                   
Процессинг файла  : экспелиармус.wav         
Распознанный текст: экс пили арбуз           
Процессинг файла  : вингардиум_левиоса.wav   
Распознанный текст: вин гарди ум леви о с а  
Процессинг файла  : алохомора.wav            
Распознанный текст: олафа мора               
Процессинг файла  : поворачивай_вправо.wav  
Распознанный текст: поворачивает вправо      
Процессинг файла  : экспекто_патронум.wav    
Распознанный текст: эксперт кто патроном     
Процессинг файла  : авада_кедавра.wav        
Распознанный текст: а вода ке дабро          
Процессинг файла  : петрификус_тоталус.wav   
Распознанный текст: петри фикус тотал ос     
Процессинг файла  : остановись.wav           
Распознанный текст: остановись               
Процессинг файла  : включи_автоматический_режим.wav
Распознанный текст: включи 

### Вторая часть. Проверка распознавания голосовых команд записанных через микрофон

In [10]:
# Загрузка дополнительных библиотек
!apt-get update
!apt-get install portaudio19-dev
!pip install vosk pydub pyaudio

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [969 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,070 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,238 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com

In [ ]:
!dpkg -s libasound2-dev

Package: libasound2-dev
Status: install ok installed
Priority: optional
Section: libdevel
Installed-Size: 677
Maintainer: Ubuntu Developers <ubuntu-devel-discuss@lists.ubuntu.com>
Architecture: amd64
Multi-Arch: same
Source: alsa-lib
Version: 1.2.6.1-1ubuntu1
Provides: libasound-dev
Depends: libasound2 (= 1.2.6.1-1ubuntu1)
Suggests: libasound2-doc
Description: shared library for ALSA applications -- development files
 This package contains files required for developing software
 that makes use of libasound2, the ALSA library.
 .
 ALSA is the Advanced Linux Sound Architecture.
Homepage: https://www.alsa-project.org/
Original-Maintainer: Debian ALSA Maintainers <pkg-alsa-devel@lists.alioth.debian.org>


In [ ]:
!pip install ffmpeg-python

In [11]:
# Загрузка дополнительных библиотек
#import os
#import sys
#import json
import pyaudio
import wave
#from vosk import Model, KaldiRecognizer
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

#### Java-скрипт необходимый для записи голоса с микрофона

In [37]:
AUDIO_HTML = """
<script>                                                      // создаем тег <script>, сообщающий браузеру о том, что внутри находится исполняемый код JavaScript
var my_div = document.createElement("DIV");                   // создаем новый элемент DIV(тег-контейнер для логического выделения блока документа)
var my_p = document.createElement("P");                       // создаем новый элемент P(параграф для логической группировки текста)
var my_btn = document.createElement("BUTTON");                // создаем новый элемент(кнопку) BUTTON
var t = document.createTextNode("Нажмите старт для записи");  // создаем текстовое содержимое для кнопки

my_btn.appendChild(t);                                        // добавляем текстовое содержимое элементу BUTTON
my_div.appendChild(my_btn);                                   // кнопку с текстом BUTTON добавляем в блок DIV
document.body.appendChild(my_div);                            // добавляем наш блок в элемент <body>("тело", для хранения содержимого веб-страницы)

var base64data = 0;                                           // будем использовать для аудиоданных метод кодирования информации в 64-разрядный код
var reader;                                                   // создаем переменную для чтения файла
var recorder, gumStream;                                      // объявляем переменные для записи данных/потока
var recordButton = my_btn;                                    // создаем переменную для кнопки записи аудио с микрофона

var handleSuccess = function(stream) {                        // объявляем функцию для работы с потоками данных
  gumStream = stream;                                         // создаем переменную для потока
  var options = {
    mimeType : 'audio/webm;codecs=opus'                       // в опциях задаем медиа тип с аудиоформатом и кодеками
  };
  recorder = new MediaRecorder(stream);                       // создаем новый объект MediaRecorder, получающий медиапоток для записи.
                                                              // MediaRecorder - интерфейс MediaStream Recording API представляющий функциональность для простой записи медиа. Создается..
                                                              // ..с использованием MediaRecorder() конструктора.
  recorder.ondataavailable = function(e) {                    // вызываем обработчик dataavailable события, запускаемое по окончанию записи
    var url = URL.createObjectURL(e.data);                    // этим методом создаем DOMString(UTF-16 String), содержащий URL с указанием на объект e.data
    var preview = document.createElement('audio');            // создаем элемент-тег аудио
    preview.controls = true;                                  // активизируем элементы управления
    preview.src = url;                                        // берем в кач-ве исходных данных файл, содержащийся в записанной ранее URL
    document.body.appendChild(preview);                       // добавляем элемент аудио в <body>("тело", для хранения содержимого веб-страницы)

    reader = new FileReader();                                // создаем объект класса FileReader для чтения разных источников данных
    reader.readAsDataURL(e.data);                             // читаем содержимое указанного файла
    reader.onloadend = function() {                           // обработчик события, запускаемого после передачи данных
      base64data = reader.result;                             // записываем прочитанное содержимое в base64data
    }
  };
  recorder.start();  // начало записи медиа
  };

// такой текст будет на кнопке BUTTON во время записи аудио
recordButton.innerText = "Идёт запись... нажмите для остановки";

// запрос разрешения пользователя на доступ к устройству захвата аудио(микрофон), указываем True
navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {                                  // функция опишет действия по завершению записи (после клика мышкой по кнопке "Recording... press to stop")
  if (recorder && recorder.state == "recording") {            // если рекордер находится в процессе записи
      recorder.stop();  // рекордер прерывается
      gumStream.getAudioTracks()[0].stop();                   // отключается запись и доступ к микрофону
      recordButton.innerText = ""    // эта надпись(сохранение записи) отобразится на кнопке BUTTON
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {                                          // создаем функцию с задержкой вызова
  return new Promise(resolve => setTimeout(resolve, ms));

  // new Promise - конструкция для отложенных вычислений
  // setTimeout позволяет вызвать функцию один раз через определённый интервал времени
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{      // при нажатии левой кнопкой мыши на кнопку "Recording... press to stop"
toggleRecording()                 // вызывается функция завершения аудиозаписи

sleep(2000).then(() => {          // и после задержки 2000мс(2 сек)
  resolve(base64data.toString())  // полученные данные из формата base64 преобразовываем в строку

});

}
});

</script>
"""

#### Функция записи звука через микрофон (в виде numpy-массива)

In [35]:
def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

#### Проверка работоспособности записи через микрофон

In [39]:
audio, sr = get_audio()

#### Функция, объединяющая запись через микрофон и распознавание голоса

In [72]:
# Функция распознавания голосовых команд через микрофон
def recognize_voice_command_mic():
    # Загрузка аудио через микрофон
    audio, sr = get_audio()
    # Преобразование numpy-массива в объект AudioSegment
    audio_segment = AudioSegment(
                                audio.tobytes(),
                                frame_rate=sr,
                                sample_width=audio.dtype.itemsize,
                                channels=1
                                )
    # Преобразование записи в один канал с частотой дискретизации 16 kHz (требование модели Vosk)
    audio = audio_segment.set_frame_rate(16000).set_channels(1)

    # Распознавание голоса
    results = []
    chunk_size_ms = 4000  # Обработка потока чанками в 4 секунды
    for i in range(0, len(audio), chunk_size_ms):
        chunk = audio[i:i + chunk_size_ms]
        if len(chunk) > 0:
            raw_data = chunk.raw_data
            recognizer.AcceptWaveform(raw_data)
            result = json.loads(recognizer.Result())
            results.append(result)

    # Сборка распознанных фрагментов и вывод на экран
    recognized_text = " ".join(result['text'] for result in results)
    print(f"Распознанный текст: {recognized_text}")


In [76]:
recognize_voice_command_mic()

Распознанный текст: на сегодня все задачи выполнены иду гулять с собакой
